The products include L1 and L2 data. In L2 we have emissivity and LST. Here are some details from the text files associated with the HyTAS products:

### 1. Level 1A Data File
Example File Name: 2014-07-06.194610.SaltonSea.Line2-Run1-Segment01-110000-level_1a.dat

This file contains calibrated HyTES data in radiance units of W/m^2/µm/sr. The data is recorded with a band interleaved by pixel (BIP) format and contains 32-bit floats with the dimensions:
  - 495 samples for data in 2013, 512 samples for data from 2014 and later.
  - A variable number of lines
  - 256 bands from 7.5-12 µm
  - All the specific dimensions for each file are described in its associated header (.hdr) file.

Header Files
Example File Name: 2014-07-06.194610.SaltonSea.Line2-Run1-Segment01-110000-level_1a.hdr
- These text header files specify the parameters of their corresponding .dat file in an human-readable and ENVI-compatible format.

### 2. Level 2 Products HDF5 File
Example File Name: 2014-07-06.194610.SaltonSea.Line2-Run1-Segment01.L2.hdf5
- Level 2 Products are not available for all lines, although most lines from 2014 and later are available.
- This file has several attributes of useful data including 'product_version' and 'acquisition_time'
- The data are arrays of 32-Bit Float values corresponding to each HyTES pixel and the arrays are shuffled and compressed with GZIP to reduce file size. The HDFs also include Fletcher32 checksums to ensure data integrity. These features are seamlessly supported by standard HDF5 readers.
- This file contains several arrays of data representing the Level 2 products as described below:

#### 2.1. Emissivity Data

This array contains HyTES emissivity spectral data from 8-11.5 µm with the same dimensions as the Level 1A data except it only has the 202 TES 'window bands' instead of 256. This is the standard emissivity product derived using the Temperature Emissivity Separation (TES) algorithm (Gillespie et al., 1998) with an In-Scene Atmospheric Correction (ISAC) approach for HyTES channels from 8-11.5 µm (only clear window channels and well-calibrated data). Spectral emissivity from 7.4-8 µm cannot be accurately derived due to strong water vapor absorption features in this spectral domain. TES is not applied to channels above 11.5 µm due to issues with calibration.

#### 2.2. PC Regression (PCemis) Emissivity Data

This array contains HyTES emissivity data from 7.4-12 µm with same dimensions as the Level 1A data. In order to produce emissivity for all HyTES bands (e.g. to be used as first guess for retrieving methane at 7.6 µm) we generated a separate PCemis product called L2.PCemis that uses a Principal Component (PC) eigenvector regression approach to produce emissivity for all HyTES channels. The eigenvectors were calculated using a set of ~150 lab spectra similar to the approach used by U. Wisconsin to produce the MODBF product (Seemann et al., 2008). The L2.PCemis produces a smoother spectrum that will fit all the original L2.emiss data, and extend this data below 8 micron and above 11.5 micron, but caution should be used when using this emissivity to identify geologic features because some features may be smoothed out, or absent.

#### 2.3. Land Surface Temperature Data File

This file contains HyTES Land Surface Temperature (LST) data in units of Kelvin with same dimensions as the Level 1A data except only one band (temperature in Kelvin). Level 2 LST Products are derived from atmospherically corrected level-1 radiance data using the TES algorithm.

Check out this site for reading data
https://lpdaac.usgs.gov/resources/e-learning/working-daily-nasa-viirs-surface-reflectance-data/

### 3. HyTAS specifications

Check the details here: https://hytes.jpl.nasa.gov/specifications

In [3]:
import h5py
import os
import matplotlib.pyplot as plt
from spectral import *
import spectral.io.envi as envi
import os
import numpy as np
import pandas as pd
import geopandas as gp
%matplotlib inline

There is a .hdr (which seems like a ENVI header file), a .dat file and two HDF5 files, one each of L1 and L2 products. 

In medatadata there four bands. Two of these are latitude and longitude of the centroid of each pixel. The third is altitude in meters and the fourth is Path Length in meters. 

In [4]:
# Functions

def geoInfo (hdrFile):
    """
    Geolocation information of measurements
    
    Parameters
    -----------
    (1) hdrFile: string
    Name of the ENVI header (.hdr) file obtained from the HyTAS site. The associated .geo.dat file should be in the same directory.
    
    Returns
    ----------
    (1) metadata (Dictionary): Metadata as a dictionary
    (2) geodata (Labelled numpy array): The four variables: Latitude, Longitude, Altitude, and Path Length
    """
    geo = envi.open(hdrFile)
    metadata = geo.metadata
    lat = geo.read_band(0)
    long = geo.read_band(1)
    altitude = geo.read_band(2)
    pathlength = geo.read_band(3)
    
    geodata = np.array([(lat), (long), (altitude), (pathlength)], dtype = [('Latitude', 'float32'), ('Longitude', 'float32'), ('Altitude', 'float32'), ('PathLength', 'float32')])
    return metadata, geodata

def listDatasets(hdf5File):
    """
    Returns a list of datasets in a HyTAS HDF file
    
    Parameters:
    -----------
    (1) hdf5File: string
    Name of the HDF5 file obtained from the HyTAS site.
    """
    
    tmp = h5py.File(hdf5File, mode='r')
    datasets = list(tmp.keys())
    return datasets

def retrieveDataset(hdf5File, hdrFile, datasetName):
    """
    Returns a dataset from the HyTAS HDF5 file
    
    Parameters:
    -----------
    (1) hdf5File: string
    Name of the HDF5 file obtained from the HyTAS site.
    
    (2) hdrFile: string
    Name of the .hdr file obtained from the HyTAS site. This should be the file that mataches the HDF5 file that holds data. 
    
    (3) datasetName: A list of strings
    The dataset(s) that you want to retrieve
    
    Returns
    ----------
    (1) A hdf5 dataset containing datasets listed in the datasetName
    
    """
    
    # Get the size of data
    metadata = geoInfo(hdrFile)[0]
    samples = int(metadata.get('samples'))
    lines = int(metadata.get('lines'))
    
    #Create empty numpy array
    
    dataArray = np.empty(shape = (len(datasetName), lines, samples), dtype = 'float32')
    
    tmp = h5py.File(hdf5File, mode='r')
    
    for i in range(len(datasetName)):
        dataset = tmp[datasetName[i]]
        dataArray[i,:,:] = dataset
    
    return (dataArray)
 
def retrieveDatasetAlt(hdf5File, datasetName):
    """
    Returns a dataset from the HyTAS HDF5 file. I am making this function because retrieveDataset, the main function, 
    can return error because the lines and samples returned by the .hdr file does not match with the size of the dataset. THIS SHOULD NOT HAPPEN.
    So, use this function only if the 'retrieveDataset' errors 
    
    Parameters:
    -----------
    (1) hdf5File: string
    Name of the HDF5 file obtained from the HyTAS site.
       
    (2) datasetName: A string
    The name of the dataset that you want to retrieve
    
    Returns
    ----------
    (1) A hdf5 dataset
    
    """
    
    tmp = h5py.File(hdf5File, mode='r')
    
    dataset = tmp[datasetName]       # Without parentheses it is a h5py dataset, not it is a numpy array
    return (dataset)    

In [6]:
# Get information of datasets in the HDF5 file
hytes = h5py.File('20190912t164958_RiversideCA_L2.hdf5', mode='r')
print('Datasets in the file: {}'.format(list(hytes.keys())))
hytes.close()

Datasets in the file: ['ISAC_Path_Rad', 'ISAC_Skydown_Rad', 'L2_Emissivity', 'L2_Emissivity_PC', 'L2_Emissivity_Wavelengths', 'L2_Emissivity_Wavelengths_PC', 'L2_LST', 'Transmission']


In [8]:
hytes = retrieveDatasetAlt('20190912t164958_RiversideCA_L2.hdf5', 'L2_LST')
print('Shape is {}'.format(hytes.shape))
print('Type is {}'.format(type(hytes)))

# Convert to numpy array
hytes = np.array(hytes[:])
print('Type now is {}'.format(type(hytes)))
print('Shape if {}'.format(hytes.shape))

Shape is (14340, 512)
Type is <class 'h5py._hl.dataset.Dataset'>
Type now is <class 'numpy.ndarray'>
Shape if (14340, 512)


Let us save as a GeoTIFF so that we can use it for plotting and analysis. 

In [10]:
latitude = np.squeeze(geodata[0,:,:])
longitude = np.squeeze(geodata[1,:,:])


print(latitude[1,1])      # It is a tuple of 4 with same corrdinate location repeated
print(longitude[1,1])     # It is a tuple of 4 with same corrdinate location repeated


# Extract one coordinate to make a point layer
latitude1 = np.empty((14327, 512))
longitude1 = np.empty((14327, 512))
for i in range(0,14327,1):
    for j in range(0, 512, 1):
        tmp = latitude[i,j]
        latitude1[i,j] = tmp[0]
        tmp = longitude[i,j]
        longitude1[i,j] = tmp[0]

(33.784634, 33.784634, 33.784634, 33.784634)
(-116.57019, -116.57019, -116.57019, -116.57019)


In [21]:
df = pd.DataFrame({'LST': hytes.flatten(), 'longitude': longitude1.flatten(), 'latitude': latitude1.flatten()})

# Convert to geodataframe
pointDf = gp.GeoDataFrame(df['LST'], geometry=gp.points_from_xy(x = df.longitude, y = df.latitude), crs="EPSG:4326")
pointDf.head()

#pointDf.to_file('/Users/manishve/Desktop/riverside.shp')

,LST,geometry
0,315.745514,POINT (-116.57059 33.78431)
1,315.520660,POINT (-116.57019 33.78463)
2,317.767639,POINT (-116.56985 33.78491)
3,318.059143,POINT (-116.56953 33.78518)
4,315.059143,POINT (-116.56921 33.78544)


Create polygon geometry, assume that the lat long points are mid points of each pixel. Convert to projected coordinate system. Riverside is in zone 6 of California's state plane systems. It is EPSG:26946.